# Basic Levelness Annotation Creation for a Crowd Sourced Gold Standard

BL annotation could range from 0-1
0, not basic level
1, surely basic level
anything inbetween gives estimate to bl likelihood

Steps:

* DF per 'annotator', create hypothetical binary annotation (bl, or not) 
 - select fastest times in both true/false type triples
 - is same category + accurate: annotate with 1
 - is different categories for fastest times + accurate: both get 0.5
 - is fastest time inaccurate, select next fastest time, give 0.5
 - two fastest times inaccurate, select remaining, give 0.3
 - all inaccurate, select superordinate, give 0.1

* Create new GOLD STANDARD data frame (per branch) like hollink (compatibility):
  -> | synset | glossary | depth in hierarchy | direct hypernyms | direct hyponyms | accuracy | blness |

* Per synset, combine blness measure (mean?), add mean accuracy
* I would still be interested in an inter-annotator measure that reveals groups of 'similarly' rating annotators. Could use to split gold standard in two w.r.t. groups of ppl.


## Data Preparation

Import Experiment Results.

Create data frame to be annotated (with correct WordNet synsets)

-> | synset | gloss | hierarchy_depth | expert_certainty | blness | accuracy |

In [2]:
from nltk.corpus import wordnet as wn
import pandas as pd
import os, csv

# specify experiment results folder in question
experiment_data_folder = os.path.abspath("../Past_experiments/Pilot2/collected_data/experiment_data")

# specify experiment 'project file'
experiment_project = pd.read_csv(os.path.abspath("../Past_experiments/Pilot2/Rosch7_Thesis_Experiment/Thesis_Experiment.csv"))

# all superordinate categories
branches = experiment_project["hypernym"].unique()

experiment_project.head()


,hypernym,bl,bl_name,bl_certainty,hyponym,hypernym_def,bl_def,hyponym_def,hyponym_img,cont_flag
0,hand_tool.n.01,awl.n.01,awl,True,bradawl.n.01,missing,a pointed tool for marking surfaces or for pun...,an awl for making small holes for brads or sma...,bradawl.n.01.png,True
1,hand_tool.n.01,file.n.04,file,True,nailfile.n.01,missing,a steel hand tool with small sharp teeth on so...,a small flat file for shaping the nails,nailfile.n.01.png,False
2,hand_tool.n.01,hammer.n.02,hammer,True,carpenter's_hammer.n.01,missing,a hand tool with a heavy rigid head and a hand...,a hammer with a cleft at one end for pulling n...,carpenter's_hammer.n.01.png,False
3,hand_tool.n.01,can_opener.n.01,can_opener,True,church_key.n.01,missing,a device for cutting cans open,can opener that has a triangular pointed end t...,church_key.n.01.png,False
4,hand_tool.n.01,plane.n.05,plane,True,smooth_plane.n.01,missing,a carpenter's hand tool with an adjustable bla...,a small plane for finish work,smooth_plane.n.01.png,False


In [3]:
def get_depth_wn(syns):
    # to do
    return 1

In [22]:
#collect array of branches as names of data frames
brs = []
syns = []
gloss = []
ecert = []
bls_old = []
bls = []
accs = []

# writing the superordinate category row

for branch in branches:
     brs.append(branch)
     syns.append(branch)
     gloss.append(wn.synset(branch).definition())
     ecert.append(True)
     bls_old.append(0)
     bls.append(0)
     accs.append(1)

for i, row in experiment_project.get(['hypernym', 'bl', 'hyponym', 'bl_certainty']).iterrows():
     bl = row['bl']
     sub = row['hyponym']
     br = row['hypernym']

     # writing the bl category row
     brs.append(br)
     syns.append(bl)
     gloss.append(wn.synset(bl).definition())
     ecert.append(certainty)
     bls_old.append(1)

     if certainty:
          bls.append(1)
          accs.append(1)
     else:
          bls.append(0.7)
          accs.append(0.5)

     # writing the subordinate category row
     brs.append(br)
     syns.append(sub)
     gloss.append(wn.synset(sub).definition())
     certainty = row['bl_certainty']
     ecert.append(certainty)
     bls_old.append(0)

     if certainty:
          bls.append(0)
          accs.append(1)
     else:
          bls.append(0.3)
          accs.append(0.5)


d = {'branch': brs,
     'synset': syns,
     'glossary': gloss,
     'expert_certainty': ecert,
     'old_bl_label': bls_old,
     'blness': bls,
     'accuracy': accs}


gold_standard_without_reaction_times = pd.DataFrame(data=d).sort_values(by=['synset'])
gold_standard_without_reaction_times.to_csv("../Past_experiments/Pilot2/computed_data/Gold_Standard_without_rt.csv")
print(len(gold_standard_without_reaction_times.index))

137


In [67]:
participant_files = [os.path.join(experiment_data_folder, file) for file in os.listdir(experiment_data_folder)]

for i in range(1,len(participant_files)+1):
    gold_standard_with_reaction_times[f'p{i}_rt'] = [-1 for x,r in gold_standard_without_reaction_times.iterrows()]
    gold_standard_with_reaction_times[f'p{i}_acc'] = [-1 for x,r in gold_standard_without_reaction_times.iterrows()]

p_id = 0

for file in participant_files:
    p_id += 1
    rt_col = f"p{p_id}_rt"
    acc_col = f"p{p_id}_acc"

    part_df = pd.read_csv(file, sep=" ",names=['index','synset','lvl', 'img', 'type', 'accuracy', 'rt'])
    part_df['accuracy'] = abs(part_df['accuracy']-2)
    part_df['synset'] = part_df['synset'].transform(lambda x: x[:-5])
    part_df = part_df.get(["synset", 'accuracy', 'rt']).sort_values(by=['synset'])
    

    for i, row in gold_standard_without_reaction_times.iterrows():
        syn = row['synset']
        strip_syn = syn[:-5].replace("'", '').replace("-", '')
        matching = part_df.loc[part_df['synset'] == strip_syn]

        # just to make sure that no synset is overlooked because of formatting
        if(len(matching)==0):
            print('alarm')
            print(strip_syn)

        if(len(matching)>2):
            print(matching)
        


    break



       synset  accuracy    rt
345  clothing         1   484
316  clothing         1   420
124  clothing         1   429
301  clothing         1   397
295  clothing         1   569
6    clothing         1   949
391  clothing         1   398
46   clothing         1   482
275  clothing         1   525
182  clothing         1   579
320  clothing         1   466
372  clothing         1   573
261  clothing         1   506
244  clothing         1   606
33   clothing         1   597
32   clothing         1   508
219  clothing         1   509
376  clothing         1   515
239  clothing         1   459
238  clothing         1   460
233  clothing         1   566
194  clothing         1   379
114  clothing         1   711
317  clothing         1   478
97   clothing         1   533
334  clothing         1  1054
98   clothing         1   664
79   clothing         1   472
93   clothing         1   512
86   clothing         1   420
           synset  accuracy   rt
41   edible_fruit         1  374
147 